In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tkan import TKAN, BSplineActivation, FixedSplineActivation
from  Kan_Layer import *
import time
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torchmetrics.regression import MeanAbsolutePercentageError, R2Score,MeanSquaredError,MeanAbsoluteError
np.random.seed(142)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import time

/Users/ange-clementakazan/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:

Abj_dt = pd.read_csv('/Users/ange-clementakazan/Documents/TKAN/Abidjan_data.csv',parse_dates={'datetime': ['YEAR', 'MO','DY']})
# Abj_dt
#Rename Columns
col_name_map = {
    "ALLSKY_SFC_SW_DWN": "ASWD",
    "CLRSKY_SFC_SW_DWN":"CSWD","ALLSKY_SFC_LW_DWN":"ALWD","PRECTOTCORR":"PREC"
}
# new_colnames=['datetime', 'ASWD', 'CSWD',
#        'ALWD', 'T2M', 'T2MDEW', 'T2MWET', 'QV2M', 'RH2M',
#        'PREC', 'PS']
# Rename the columns for both datasets
Abj_dt=Abj_dt.set_index("datetime")
Abj_dt=Abj_dt.rename(columns=col_name_map)
# Xy_train
# target_col = Abj_dt.pop("T2M")
# Abj_dt.insert(len(Abj_dt.columns),"T2M", target_col)
# Abj_dt
target_col = Abj_dt.pop("PREC")
Abj_dt.insert(len(Abj_dt.columns),"PREC", target_col)
Abj_dt

/var/folders/nv/ct8x_h9j78xbj8gnyb35c3_w0000gn/T/ipykernel_80973/2212243777.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  Abj_dt = pd.read_csv('/Users/ange-clementakazan/Documents/TKAN/Abidjan_data.csv',parse_dates={'datetime': ['YEAR', 'MO','DY']})


,ASWD,CSWD,ALWD,T2M,T2MDEW,T2MWET,QV2M,RH2M,PS,PREC
datetime,,,,,,,,,,
2010-01-01,4.06,5.86,412.57,28.05,22.73,25.39,17.33,74.56,100.44,0.08
2010-01-02,5.21,5.58,408.32,28.34,22.48,25.41,17.09,72.56,100.45,0.05
2010-01-03,4.77,5.38,412.82,28.69,22.51,25.60,17.15,71.75,100.41,0.08
2010-01-04,5.17,5.55,407.15,28.44,22.46,25.45,17.15,73.00,100.43,0.05
2010-01-05,5.26,5.57,406.60,28.20,22.46,25.33,17.09,73.50,100.43,0.06
...,...,...,...,...,...,...,...,...,...,...
2023-12-28,4.47,5.38,401.95,27.37,22.82,25.09,17.46,77.75,100.65,0.15
2023-12-29,4.44,5.26,410.70,27.99,24.05,26.01,18.74,80.06,100.55,0.17
2023-12-30,5.12,5.40,409.38,27.72,24.11,25.91,18.80,81.94,100.53,0.26


In [3]:
dates=Abj_dt.index.astype(str).tolist()

In [4]:
# scaler = MinMaxScaler(feature_range=(0,1))
# def Make_tidydata(data,scaler):
#     data= data.sort_index()
#     data[data.columns] = scaler.fit_transform(data[data.columns])
#     # Splitting the dataset into training, validation, and test sets
#     train_size = int(len(data)*0.8)
#     val_size =int(train_size*0.10)
#     #Train_val_test sets
#     Xy_train = data[:train_size-val_size]
#     Xy_val= data[train_size-val_size:train_size]
#     Xy_test=data[train_size:]
#     return Xy_train,Xy_val,Xy_test
# Xy_train,Xy_val,Xy_test=Make_tidydata(Abj_dt,scaler)


In [5]:
#################### Datapreprocessing For Temperature   #####################

# Sorting the dates
def Make_tidydata(data,scaler):
    data= data.sort_index()
    data[data.columns] = scaler.fit_transform(data[data.columns])
    # Splitting the dataset into training, validation, and test sets
    train_size = int(len(data)*0.8)
    val_size =int(train_size*0.10)
    #Train_val_test sets
    Xy_train = data[:train_size-val_size]
    Xy_val= data[train_size-val_size:train_size]
    Xy_test=data[train_size:]
    return Xy_train,Xy_val,Xy_test

### Define the TimeSeriesDataset class ###
## Define the TimeSeriesDataset class
class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe, n_steps_in, n_steps_out,target_names):
        """
        Initialize the dataset with a pandas DataFrame.
        Parameters:
        - dataframe: Pandas DataFrame containing the time series data.
        - n_steps_in: Number of time steps for input.
        - n_steps_out: Number of time steps for output.
        -target_names: Names of the target variables.
        """
        self.dataframe = dataframe
        self.n_steps_in = n_steps_in
        self.n_steps_out = n_steps_out   
        self.target_names=target_names   
        # Prepare the input and output sequences
        self.X, self.y = self.sliding_window(self.dataframe, self.n_steps_in, self.n_steps_out,self.target_names)
    def sliding_window(self, dataframe, n_steps_in, n_steps_out,target_names):
        X, y = list(), list()  
        for i in range(len(dataframe)):
            end_idx = i + n_steps_in
            out_end_idx = end_idx + n_steps_out - 1
            # Ensure valid index range
            if out_end_idx >= len(dataframe):
                break
            # Extract X and y sequences
            seq_x = dataframe.iloc[i:end_idx, :].values  # X sequence
            seq_y = dataframe[target_names][end_idx:out_end_idx + 1].values  # y sequence (last column)| Redefine this!!!
            # Check if `seq_y` is shorter than `n_steps_out`
            if len(seq_y) < n_steps_out:
                pad_length = n_steps_out - len(seq_y)
                avg_value = np.mean(seq_y)  # Calculate the average of `seq_y`
                # Pad `seq_y` with the average value
                seq_y = np.pad(seq_y, (0, pad_length), 'constant', constant_values=avg_value)
            # Ensure `seq_y` is a 2D array
            if len(seq_y.shape) == 1:
                seq_y = seq_y.reshape(-1, 1)
            # Append the sequences to the list
            X.append(seq_x)
            y.append(seq_y)
        return np.array(X), np.array(y)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        # Return the input-output pair for the given index
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)
    
# Create data loaders
def create_data_loaders(TimeSeriesDataset,Make_tidydata,df,target_names,scaler,window_length,obs_length,batch_size):
    Xy_train,Xy_val,Xy_test=Make_tidydata(df,scaler)
    # Creating time series data
    train_dataset = TimeSeriesDataset(Xy_train,obs_length, window_length,target_names)
    val_dataset = TimeSeriesDataset(Xy_val, obs_length, window_length,target_names)
    test_dataset = TimeSeriesDataset(Xy_test, obs_length, window_length,target_names)
    # Creating data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader,val_loader,test_loader

# Create DataLoaders for Abidjan and Kigali
batch_size = 64
window_length = 1
obs_length=14
scaler=[MinMaxScaler(feature_range=(0,1)),MinMaxScaler(feature_range=(0,1))]
target_names=['T2M'] #,'PREC','PS'
Abj_train_loader,Abj_val_loader,Abj_test_loader=create_data_loaders(TimeSeriesDataset,Make_tidydata,Abj_dt,target_names,scaler[0],window_length,obs_length,batch_size)
# Kgl_train_loader,Kgl_val_loader,Kgl_test_loader=create_data_loaders(TimeSeriesDataset,Make_tidydata,Kgl_dt,target_names,scaler[1],window_length,obs_length,batch_size)

In [6]:
def dates_splitting(date):
    # Splitting the dataset into training, validation, and test sets
    train_size = int(len(date)*0.8)
    val_size =int(train_size*0.10)
    #Train_val_test sets
    date_train = date[:train_size-val_size]
    date_val= date[train_size-val_size:train_size]
    date_test=date[train_size:]
    return date_train,date_val,date_test

In [7]:
# # Define the data 
# X_train,y_train=Xy_train.values[:,:-1],Xy_train.values[:,-1]
# X_train = torch.tensor(X_train, dtype=torch.float32)  # Convert to float32
# y_train = torch.tensor(y_train, dtype=torch.float32)  # Convert to float32

In [8]:
# import torch
# import torch.optim as optim
# import torch.nn as nn
# from sklearn.datasets import make_regression
# from sklearn.preprocessing import StandardScaler
# from torch.utils.data import DataLoader, TensorDataset


# # Create DataLoader
# batch_size = 32
# train_dataset = TensorDataset(X_train, y_train.reshape(-1, 1))
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [9]:

# # Пример данных
# inputs = torch.rand((32, 10, 8))  # batch_size=32, sequence_length=10, input_size=8
#targets = torch.randint(0, 2, (32, 4)).float()  # Пример целей для обучения
input_shape=Abj_dt.shape
sub_kan_input_dim=input_shape[-1]
sub_kan_output_dim=sub_kan_input_dim
units=10
# sub_kan_input_dim

# tkan_layer 
model= TKAN(units, input_shape, sub_kan_output_dim, sub_kan_input_dim, 
                  tkan_activations=[BSplineActivation(3), FixedSplineActivation(2.0)])
# model = nn.Sequential(tkan_layer, nn.Flatten(), nn.Linear(14, 1), nn.Sigmoid()) 


# Setting up the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
#criterion = nn.BCELoss()
criterion = nn.MSELoss()


# # Training loop

model.train()
epochs = 100
for epoch in range(epochs): 
    epoch_loss = 0
    for X_batch, y_batch in Abj_train_loader:
        optimizer.zero_grad()   # Reset gradients
        print("Shape of Xbatch:",X_batch.shape)
        outputs, _ = model(X_batch)
        outputs = outputs[:, -1, :]  
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        if epoch % 10 == 0:  # Print loss every 10 epochs
            print(f"Epoch [{epoch}/{epochs}], Loss: {epoch_loss / len(Abj_train_loader)}") #:.4f
        
print("Training complete! 🚀")



Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.0056947813979510604
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.00923937584819465
Shape of Xbatch: torch.Size([64, 14, 10])


/Users/ange-clementakazan/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64, 1, 1])) that is different to the input size (torch.Size([64, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [0/100], Loss: 0.009903973289604845
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.010112065436511204
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.010978121246243346
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.012465669484487895
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.014359094523664179
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.01598856090728579
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.01625089348968247
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.016574949652341932
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.01767964325524096
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.020226753975168384
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.022794416479380995
Shape of Xbatch: torch.Size([64, 14, 10])
Epoch [0/100], Loss: 0.023601520983181124
Shape of Xbatch: torch.Size([64, 14, 

/Users/ange-clementakazan/anaconda3/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([20, 1, 1])) that is different to the input size (torch.Size([20, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 14, 10])
Shape of Xbatch: torch.Size([64, 1

KeyboardInterrupt: 

In [ ]:
# # Тестирование модели
# model.eval()
# with torch.no_grad():
#     test_inputs = torch.rand((10, 10, 8))  # Пример тестовых данных
#     test_outputs, _ = tkan_layer(test_inputs)
#     test_outputs = test_outputs[:, -1, :]
#     print("Test Outputs:", test_outputs)

In [10]:
import torch
from spline import BSplineActivation

# Define model parameters
batch_size = 64
seq_len = 10
input_dim = 21  # Must match input_shape[-1]
units = 32  # Hidden size
sub_kan_output_dim = 4
sub_kan_input_dim = 10
input_shape = (batch_size, seq_len, input_dim)

# Generate random input data
dummy_input = torch.randn(batch_size, seq_len, input_dim)

# Instantiate the model
tkan_model = TKAN(units, input_shape, sub_kan_output_dim, sub_kan_input_dim, [BSplineActivation(3)])

# Run the model
output, final_states = tkan_model(dummy_input)

print("Output Shape:", output.shape)  # Expected: (batch_size, seq_len, units)
for i, state in enumerate(final_states):
    print(f"State {i} Shape:", state.shape)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x21 and 32x96)